In [20]:
import pandas as pd
import pyodbc
import sqlalchemy
from time import time
from main import odbc_connection
from main import odbc_url_sqlalchemy

In [21]:
def time_measure(function):
    def measure_function(*args, **kwargs):
        start = time()
        c = function(*args, **kwargs)
        print(time() - start)
        return c
    return measure_function

In [22]:
@time_measure
def read_sql_pandas(query: str, connection: pyodbc.Connection) -> pd.DataFrame:
    """
    :param query:
    :param connection:
    :return: pd.Dataframe
    """

    try:
        df = pd.read_sql(query, connection)
        return df

    except OSError as e:
        raise OSError(f'Error creating dataframe from database {e}')

    finally:
        connection.close()

In [23]:

query_atypical_date = 'SELECT * FROM [ETLStaging].[AtypicalDate]'
connection_db = odbc_connection()
data_frame = read_sql_pandas(query_atypical_date, connection_db)

data_frame

/var/folders/sh/f6gkgjfs5lx3pkj9hkl44vgc0000gn/T/ipykernel_28700/715247001.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


1.047248125076294


,AtypicalDate,AtypicalDateCode,AtypicalDateName,TypeDay,XMCreationDate
0,2000-01-01,1--ENE,Primero de enero,FEST,2020-02-04 18:31:22
1,2000-01-02,2--ENE,Dos de enero,ORD,2020-02-04 18:31:22
2,2000-01-03,LUNES,Lunes en vacaciones de enero,ORD,2020-02-04 18:31:22
3,2000-01-04,MARTES,Martes en vacaciones de enero,ORD,2020-02-04 18:31:22
4,2000-01-05,MIERCOLES,MiÃ©rcoles en vacaciones de enero,ORD,2020-02-04 18:31:22
...,...,...,...,...,...
18623,2050-12-27,MARTES,Martes en vacaciones de diciembre,ORD,2020-02-04 18:31:22
18624,2050-12-28,MIERCOLES,MiÃ©rcoles en vacaciones de diciembre,ORD,2020-02-04 18:31:22
18625,2050-12-29,JUEVES,Jueves en vacaciones de diciembre,ORD,2020-02-04 18:31:22
18626,2050-12-30,VIERNES,Viernes en vacaciones de diciembre,ORD,2020-02-04 18:31:22


In [24]:
@time_measure
def read_sql_sqlalchemy(query_str: str, url: str) -> pd.DataFrame:
    """
    :param query_str:
    :param url:
    :return:
    """

    try:
        engine = sqlalchemy.create_engine(url)
        df = pd.read_sql(query_str, engine)
        return df

    except OSError as e:
        raise OSError(f'Error creating dataframe from database {e}')

In [25]:
url_db = odbc_url_sqlalchemy()
df_alchemy = read_sql_sqlalchemy(query_atypical_date, url_db)

3.0364701747894287


In [26]:
df_alchemy

,AtypicalDate,AtypicalDateCode,AtypicalDateName,TypeDay,XMCreationDate
0,2000-01-01,1--ENE,Primero de enero,FEST,2020-02-04 18:31:22
1,2000-01-02,2--ENE,Dos de enero,ORD,2020-02-04 18:31:22
2,2000-01-03,LUNES,Lunes en vacaciones de enero,ORD,2020-02-04 18:31:22
3,2000-01-04,MARTES,Martes en vacaciones de enero,ORD,2020-02-04 18:31:22
4,2000-01-05,MIERCOLES,MiÃ©rcoles en vacaciones de enero,ORD,2020-02-04 18:31:22
...,...,...,...,...,...
18623,2050-12-27,MARTES,Martes en vacaciones de diciembre,ORD,2020-02-04 18:31:22
18624,2050-12-28,MIERCOLES,MiÃ©rcoles en vacaciones de diciembre,ORD,2020-02-04 18:31:22
18625,2050-12-29,JUEVES,Jueves en vacaciones de diciembre,ORD,2020-02-04 18:31:22
18626,2050-12-30,VIERNES,Viernes en vacaciones de diciembre,ORD,2020-02-04 18:31:22


In [27]:
from datetime import datetime

df_calendar = df_alchemy

def dias(df):
    df['date_tmp'] = df['AtypicalDate'].apply(
    lambda x: datetime.strptime(str(x), "%Y-%m-%d")
    )
    df['day'] = df.date_tmp.dt.day
    df['month'] = df.date_tmp.dt.month
    df['year'] = df.date_tmp.dt.year

In [28]:
dias(df_calendar)
df_calendar

,AtypicalDate,AtypicalDateCode,AtypicalDateName,TypeDay,XMCreationDate,date_tmp,day,month,year
0,2000-01-01,1--ENE,Primero de enero,FEST,2020-02-04 18:31:22,2000-01-01,1,1,2000
1,2000-01-02,2--ENE,Dos de enero,ORD,2020-02-04 18:31:22,2000-01-02,2,1,2000
2,2000-01-03,LUNES,Lunes en vacaciones de enero,ORD,2020-02-04 18:31:22,2000-01-03,3,1,2000
3,2000-01-04,MARTES,Martes en vacaciones de enero,ORD,2020-02-04 18:31:22,2000-01-04,4,1,2000
4,2000-01-05,MIERCOLES,MiÃ©rcoles en vacaciones de enero,ORD,2020-02-04 18:31:22,2000-01-05,5,1,2000
...,...,...,...,...,...,...,...,...,...
18623,2050-12-27,MARTES,Martes en vacaciones de diciembre,ORD,2020-02-04 18:31:22,2050-12-27,27,12,2050
18624,2050-12-28,MIERCOLES,MiÃ©rcoles en vacaciones de diciembre,ORD,2020-02-04 18:31:22,2050-12-28,28,12,2050
18625,2050-12-29,JUEVES,Jueves en vacaciones de diciembre,ORD,2020-02-04 18:31:22,2050-12-29,29,12,2050
18626,2050-12-30,VIERNES,Viernes en vacaciones de diciembre,ORD,2020-02-04 18:31:22,2050-12-30,30,12,2050


In [29]:
@time_measure
def rename():
    df_calendar['AtypicalDateCode'] = df_calendar.AtypicalDateCode.str.lower()
    df_calendar['AtypicalDateCode'] = df_calendar.AtypicalDateCode.str.replace('1--ene', 'ene_1')
    df_calendar['AtypicalDateCode'] = df_calendar.AtypicalDateCode.str.replace('2--ene', 'ene_2')
    df_calendar['AtypicalDateCode'] = df_calendar.AtypicalDateCode.str.replace('1--may', 'may_1')
    df_calendar['AtypicalDateCode'] = df_calendar.AtypicalDateCode.str.replace('20--jul', 'jul_20')
    df_calendar['AtypicalDateCode'] = df_calendar.AtypicalDateCode.str.replace('7--ago', 'ago_7')
    df_calendar['AtypicalDateCode'] = df_calendar.AtypicalDateCode.str.replace('8--dic', 'dic_8')
    df_calendar['AtypicalDateCode'] = df_calendar.AtypicalDateCode.str.replace('24--dic', 'dic_24')
    df_calendar['AtypicalDateCode'] = df_calendar.AtypicalDateCode.str.replace('25--dic', 'dic_25')
    df_calendar['AtypicalDateCode'] = df_calendar.AtypicalDateCode.str.replace('31--dic', 'dic_31')

    return df_calendar

In [30]:
rename()

0.054342031478881836


,AtypicalDate,AtypicalDateCode,AtypicalDateName,TypeDay,XMCreationDate,date_tmp,day,month,year
0,2000-01-01,ene_1,Primero de enero,FEST,2020-02-04 18:31:22,2000-01-01,1,1,2000
1,2000-01-02,ene_2,Dos de enero,ORD,2020-02-04 18:31:22,2000-01-02,2,1,2000
2,2000-01-03,lunes,Lunes en vacaciones de enero,ORD,2020-02-04 18:31:22,2000-01-03,3,1,2000
3,2000-01-04,martes,Martes en vacaciones de enero,ORD,2020-02-04 18:31:22,2000-01-04,4,1,2000
4,2000-01-05,miercoles,MiÃ©rcoles en vacaciones de enero,ORD,2020-02-04 18:31:22,2000-01-05,5,1,2000
...,...,...,...,...,...,...,...,...,...
18623,2050-12-27,martes,Martes en vacaciones de diciembre,ORD,2020-02-04 18:31:22,2050-12-27,27,12,2050
18624,2050-12-28,miercoles,MiÃ©rcoles en vacaciones de diciembre,ORD,2020-02-04 18:31:22,2050-12-28,28,12,2050
18625,2050-12-29,jueves,Jueves en vacaciones de diciembre,ORD,2020-02-04 18:31:22,2050-12-29,29,12,2050
18626,2050-12-30,viernes,Viernes en vacaciones de diciembre,ORD,2020-02-04 18:31:22,2050-12-30,30,12,2050


In [31]:
list_date_code = df_calendar.AtypicalDateCode.unique()

In [32]:
list_date_code_2 = [
    'ene_1', 'ene_2', 'may_1', 'jul_20', 'ago_7', 'dic_8', 'dic_24', 'dic_25',
    'dic_31', 'mss', 'jss', 'vss', 'sss', 'dss', 'lf'
]

list_type_day = df_calendar.TypeDay.unique()
list_type_day

array(['FEST', 'ORD', 'SAB'], dtype=object)

In [33]:
variable_type = 'P'
measuring_element_id = 32
connection_db = odbc_connection()
query_get_measure = f"EXEC [Forecast].[usp_GetInputMeasure] @MeasuringElementMRID = {measuring_element_id}, @MeasuresTypeMRID = {variable_type};"

read_sql_pandas(query_get_measure, connection_db)

/var/folders/sh/f6gkgjfs5lx3pkj9hkl44vgc0000gn/T/ipykernel_28700/715247001.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


1.3967227935791016


,InputMeasureMRID,MeasuringDate,MeasuringElementMRID,MeasuresTypeMRID,ProcessTypeMRID,OriginTimeSeriesMRID,Resolution,MeasuringValue,XMCreationDate
0,2012131573,2022-10-14 00:00:00,32,P,20,95,5,19.5929,2022-10-14 07:06:32.030
1,2012123517,2022-10-14 00:05:00,32,P,20,95,5,19.4979,2022-10-14 07:06:30.433
2,2012127789,2022-10-14 00:10:00,32,P,20,95,5,19.2128,2022-10-14 07:06:31.160
3,2012130686,2022-10-14 00:15:00,32,P,20,95,5,19.0133,2022-10-14 07:06:31.617
4,2012123745,2022-10-14 00:20:00,32,P,20,95,5,18.8612,2022-10-14 07:06:30.467
...,...,...,...,...,...,...,...,...,...
34997,2697246316,2023-02-13 23:40:00,32,P,20,95,5,19.3838,2023-02-13 23:40:51.333
34998,2697246506,2023-02-13 23:45:00,32,P,20,95,5,18.8517,2023-02-13 23:45:52.667
34999,2697246758,2023-02-13 23:50:00,32,P,20,95,5,18.8517,2023-02-13 23:51:16.590
35000,2697246918,2023-02-13 23:55:00,32,P,20,95,5,18.8517,2023-02-13 23:55:52.640


In [34]:
url_db = odbc_url_sqlalchemy()
read_sql_sqlalchemy(query_get_measure, url_db)

3.374099016189575


,InputMeasureMRID,MeasuringDate,MeasuringElementMRID,MeasuresTypeMRID,ProcessTypeMRID,OriginTimeSeriesMRID,Resolution,MeasuringValue,XMCreationDate
0,2012131573,2022-10-14 00:00:00,32,P,20,95,5,19.5929,2022-10-14 07:06:32.030
1,2012123517,2022-10-14 00:05:00,32,P,20,95,5,19.4979,2022-10-14 07:06:30.433
2,2012127789,2022-10-14 00:10:00,32,P,20,95,5,19.2128,2022-10-14 07:06:31.160
3,2012130686,2022-10-14 00:15:00,32,P,20,95,5,19.0133,2022-10-14 07:06:31.617
4,2012123745,2022-10-14 00:20:00,32,P,20,95,5,18.8612,2022-10-14 07:06:30.467
...,...,...,...,...,...,...,...,...,...
34997,2697246316,2023-02-13 23:40:00,32,P,20,95,5,19.3838,2023-02-13 23:40:51.333
34998,2697246506,2023-02-13 23:45:00,32,P,20,95,5,18.8517,2023-02-13 23:45:52.667
34999,2697246758,2023-02-13 23:50:00,32,P,20,95,5,18.8517,2023-02-13 23:51:16.590
35000,2697246918,2023-02-13 23:55:00,32,P,20,95,5,18.8517,2023-02-13 23:55:52.640
